# Zadaca 2: Stabla odlučivanja

## Zadatak 1: Modifikacija stabla odlučivanja

Modifikovati postojeću implementaciju algoritma tako da se dodaju sljedeće stavke:
- radukovani kandidatski splitovi
- random subspace sampling
- opcioni cost-complexity prunning

Potrebno je omogućiti da upotreba dodatnih stavki bude opciona, kroz proslijeđivanje odgovarajućih vrijednosti u konstruktor. Testirati brzinu i F1 metriiku rada klasifikatora na proizvoljnom skupu podataka, sa različitim kombinacijama novih opcija.

In [1]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=5, min_size=10):
        """
        Inicijalizacija klasifikatora odluka stabla.
        :param max_depth: Maksimalna dubina stabla.
        :param min_size: Minimalan broj uzoraka potreban za podelu čvora.
        """
        self.max_depth = max_depth  # ograničenje maksimalne dubine
        self.min_size = min_size    # minimalan broj uzoraka za podelu
        self.root = None            # korjenski čvor stabla

    def fit(self, X, y):
        """
        Gradi stablo odluke na osnovu skupa za treniranje (X, y).
        :param X: Lista ulaznih vektora.
        :param y: Lista ciljnih klasa.
        """
        # Kombinujemo X i y u jedinstveni dataset (poslednji element je labela)
        dataset = [row[:] + [label] for row, label in zip(X, y)]
        # Građenje stabla počevši od korena na dubini 1
        self.root = self._build_tree(dataset, depth=1)

    def predict(self, X):
        """
        Predviđa klase za ulazne podatke X.
        :param X: Lista ulaznih vektora.
        :return: Lista predviđenih klasa.
        """
        # Rekurzivno prolazi kroz stablo za svaki red (vector)
        return [self._predict_row(self.root, row) for row in X]

    # ----- Pomoćne metode ----- #

    def _gini_index(self, groups, class_values):
        """
        Računa Gini nečistoću za podjelu.
        :param groups: Lista grupa [levo, desno] nakon podjele.
        :param class_values: Lista svih mogućih klasa.
        :return: Gini indeks.
        """
        n_instances = float(sum(len(group) for group in groups))  # ukupan broj primera
        gini = 0.0
        for group in groups:
            size = float(len(group))
            if size == 0:
                continue  # izbegavamo deljenje nulom
            score = 0.0
            for class_val in class_values:
                p = [row[-1] for row in group].count(class_val) / size
                score += p * p
            gini += (1.0 - score) * (size / n_instances)
        return gini

    def _test_split(self, index, value, dataset):
        """
        Dijeli dataset na lijevi i desni podskup po pravilu: row[index] < value.
        :param index: Indeks karakteristike po kojoj dijelimo podatke.
        :param value: Vrijednost granice podjele.
        :param dataset: Skup podataka (redovi sa labelama).
        :return: Tuple (levo, desno) lista redova.
        """
        left, right = [], []
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    def _get_best_split(self, dataset):
        """
        Pronalazi najbolju podjelu skupa podataka minimizujući Gini index.
        :param dataset: Skup podataka koji treba da se podijele.
        :return: Dictionary sa ključevima 'index', 'value', 'groups'.
        """
        class_values = list(set(row[-1] for row in dataset))
        best_index, best_value, best_score, best_groups = None, None, float('inf'), None
        for index in range(len(dataset[0]) - 1):
            for row in dataset:
                groups = self._test_split(index, row[index], dataset)
                gini = self._gini_index(groups, class_values)
                if gini < best_score:
                    best_index, best_value, best_score, best_groups = index, row[index], gini, groups
        return {'index': best_index, 'value': best_value, 'groups': best_groups}

    def _to_terminal(self, group):
        """
        Kreira terminalni čvor vraćajući najčešću klasu u grupi.
        :param group: Lista redova u čvoru.
        :return: Vrednost klase (labela).
        """
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)

    def _split(self, node, depth):
        """
        Rekurzivno dijeli čvor sve dok se ne zadovolje uslovi zaustavljanja.
        :param node: Trenutni čvor: dictionary sa ključevima 'index', 'value', 'groups'.
        :param depth: Trenutna dubina stabla.
        """
        left, right = node['groups']
        del node['groups']
        if not left or not right:
            terminal = self._to_terminal(left + right)
            node['left'] = node['right'] = terminal
            return
        if depth >= self.max_depth:
            node['left'], node['right'] = self._to_terminal(left), self._to_terminal(right)
            return
        if len(left) <= self.min_size:
            node['left'] = self._to_terminal(left)
        else:
            node['left'] = self._get_best_split(left)
            self._split(node['left'], depth + 1)
        if len(right) <= self.min_size:
            node['right'] = self._to_terminal(right)
        else:
            node['right'] = self._get_best_split(right)
            self._split(node['right'], depth + 1)

    def _build_tree(self, train, depth):
        """
        Počinje pravljenje stabla od korena.
        :param train: Dataset za treniranje.
        :param depth: Početna dubina (1).
        :return: Rečnik koji predstavlja čvor korena.
        """
        root = self._get_best_split(train)
        self._split(root, depth)
        return root

    def _predict_row(self, node, row):
        """
        Predviđa labelu za jedan ulazni vektor rekurzivno prolazeći stablo.
        :param node: Trenutni čvor stabla ili terminalna vrednost.
        :param row: Ulazni vektor.
        :return: Predviđena labela.
        """
        if isinstance(node, dict):
            if row[node['index']] < node['value']:
                return self._predict_row(node['left'], row)
            else:
                return self._predict_row(node['right'], row)
        return node